In [2]:
import time

In [7]:
time.localtime()

time.struct_time(tm_year=2022, tm_mon=8, tm_mday=28, tm_hour=21, tm_min=37, tm_sec=41, tm_wday=6, tm_yday=240, tm_isdst=0)

In [12]:
# 오전 9 - 11
# 오후 13 - 18 
from time import localtime

#TODO 매 시간마다
#TODO 평일마다
if 9 <= localtime().tm_hour <= 11 or 13 <= localtime().tm_hour <= 18:
    print('자세 바로하세요') 

In [25]:
# 년, 월, 일, 시간 출력
time.strftime("%Y년 %m월 %d일 %H:%M:%S")

'2022년 08월 28일 21:47:15'